# ANALISIS DE LA NBA

In [1]:
from pathlib import Path                  # Manejo seguro de rutas
import sqlite3                            # Conexión a SQLite (estándar en Python)
import pandas as pd                       
import numpy as np                        

In [2]:
DB_PATH = Path("nba.sqlite")   

In [ ]:
# --- Función para obtener nombres de tablas ---
def get_tables(connection):
    """
    Devuelve una lista con los nombres de las tablas en la base SQLite.
    """
    q = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
    return pd.read_sql_query(q, connection)["name"].tolist()

# --- Función para ver columnas de una tabla específica ---
def get_columns(connection, table_name):
    """
    Devuelve un DataFrame con la metadata de columnas de la tabla 'table_name'.
    Incluye: cid (índice), name (nombre), type (tipo), notnull, dflt_value, pk (si es clave primaria).
    """
    q = f"PRAGMA table_info({table_name});"  
    return pd.read_sql_query(q, connection)

# ---  Listar todas las tablas ---
tables = get_tables(conn)           
print(f"Tablas encontradas ({len(tables)}):")
for t in tables:
    print(" -", t)


Tablas encontradas (16):
 - common_player_info
 - draft_combine_stats
 - draft_history
 - game
 - game_info
 - game_summary
 - inactive_players
 - line_score
 - officials
 - other_stats
 - play_by_play
 - player
 - team
 - team_details
 - team_history
 - team_info_common


In [16]:
# --- Resumen: filas y columnas de cada tabla ---
resumen = []

for t in get_tables(conn):
    try:
        n_filas = pd.read_sql_query(f"SELECT COUNT(*) AS n FROM {t};", conn).loc[0, "n"]
        n_columnas = len(get_columns(conn, t))
        resumen.append((t, n_filas, n_columnas))
    except Exception as e:
        resumen.append((t, f"Error: {e}", None))

# Pasamos a DataFrame para verlo ordenado
df_resumen = pd.DataFrame(resumen, columns=["tabla", "filas", "columnas"])
display(df_resumen)


,tabla,filas,columnas
0,common_player_info,3632,33
1,draft_combine_stats,1633,47
2,draft_history,8257,14
3,game,65698,55
4,game_info,58053,4
5,game_summary,58110,14
6,inactive_players,110191,9
7,line_score,58053,43
8,officials,70971,5
9,other_stats,28271,26


In [20]:
import shutil

# Ruta a tu archivo original
DB_PATH = Path("nba.sqlite")   # asegúrate que este es tu archivo base

# Ruta para la copia (mismo nombre con sufijo _work)
DB_WORK_PATH = Path(DB_PATH.parent / (DB_PATH.stem + "_work" + DB_PATH.suffix))

# Crear copia
shutil.copy2(DB_PATH, DB_WORK_PATH)

print(f"✅ Copia creada en: {DB_WORK_PATH}")

✅ Copia creada en: nba_work.sqlite


In [21]:
# Cerramos conexión previa (si existía) y nos conectamos a la copia
try:
    conn.close()
except:
    pass

conn = sqlite3.connect(DB_WORK_PATH)
print("Conectado a la copia de trabajo ✅")


Conectado a la copia de trabajo ✅


# TABLA GAME 

In [22]:
def reduce_table(conn, table_name, keep_cols):
    """
    Crea una nueva tabla con solo las columnas indicadas y
    reemplaza la original en la base SQLite.
    """
    # 1) Validar que las columnas existen en la tabla
    cols_existentes = get_columns(conn, table_name)["name"].tolist()
    existentes = [c for c in keep_cols if c in cols_existentes]
    if not existentes:
        raise ValueError(f"Ninguna de las columnas pedidas existe en {table_name}")

    cols_sql = ", ".join(existentes)

    # 2) Crear tabla temporal con solo esas columnas
    tmp_table = f"{table_name}_tmp"
    conn.execute(f"DROP TABLE IF EXISTS {tmp_table};")
    conn.execute(f"CREATE TABLE {tmp_table} AS SELECT {cols_sql} FROM {table_name};")

    # 3) Borrar tabla original y renombrar la temporal
    conn.execute(f"DROP TABLE {table_name};")
    conn.execute(f"ALTER TABLE {tmp_table} RENAME TO {table_name};")
    conn.commit()

    print(f"✅ Tabla '{table_name}' reducida a {len(existentes)} columnas.")
    return existentes


In [23]:
game_cols = [
    # Identificadores
    "game_id","season_id","season_type","game_date",
    "team_id_home","team_id_away","wl_home",
    # Tiro
    "fgm_home","fga_home","fg3m_home","fg3a_home","ftm_home","fta_home",
    "fgm_away","fga_away","fg3m_away","fg3a_away","ftm_away","fta_away",
    # Rebotes
    "oreb_home","dreb_home","reb_home",
    "oreb_away","dreb_away","reb_away",
    # Otros
    "ast_home","stl_home","blk_home","tov_home","pf_home","pts_home",
    "ast_away","stl_away","blk_away","tov_away","pf_away","pts_away"
]


In [24]:
reduce_table(conn, "game", game_cols)

# Revisar resultado
get_columns(conn, "game")


✅ Tabla 'game' reducida a 37 columnas.


,cid,name,type,notnull,dflt_value,pk
0,0,game_id,TEXT,0,None,0
1,1,season_id,TEXT,0,None,0
2,2,season_type,TEXT,0,None,0
3,3,game_date,NUM,0,None,0
4,4,team_id_home,TEXT,0,None,0
5,5,team_id_away,TEXT,0,None,0
6,6,wl_home,TEXT,0,None,0
7,7,fgm_home,REAL,0,None,0
8,8,fga_home,REAL,0,None,0
9,9,fg3m_home,REAL,0,None,0


# TABLA other_stats

In [27]:
# --- Reducir 'other_stats' a las columnas acordadas y mostrar resultado + tamaño ---
import os

other_stats_cols = [
    "game_id",
    "team_id_home","team_id_away",
    "pts_paint_home","pts_paint_away",
    "pts_2nd_chance_home","pts_2nd_chance_away",
    "pts_fb_home","pts_fb_away",
    "lead_changes","times_tied",
    "team_turnovers_home","team_turnovers_away",
    "total_turnovers_home","total_turnovers_away",
    "team_rebounds_home","team_rebounds_away",
    "pts_off_to_home","pts_off_to_away"
]

reduce_table(conn, "other_stats", other_stats_cols)

# Ver columnas resultantes en 'other_stats'
display(get_columns(conn, "other_stats"))


✅ Tabla 'other_stats' reducida a 19 columnas.


,cid,name,type,notnull,dflt_value,pk
0,0,game_id,TEXT,0,None,0
1,1,team_id_home,TEXT,0,None,0
2,2,team_id_away,TEXT,0,None,0
3,3,pts_paint_home,INT,0,None,0
4,4,pts_paint_away,INT,0,None,0
5,5,pts_2nd_chance_home,INT,0,None,0
6,6,pts_2nd_chance_away,INT,0,None,0
7,7,pts_fb_home,INT,0,None,0
8,8,pts_fb_away,INT,0,None,0
9,9,lead_changes,INT,0,None,0


# line_score

In [28]:
line_score_cols = [
    "game_id",
    "team_id_home","team_id_away",
    # Puntos por cuartos (home)
    "pts_qtr1_home","pts_qtr2_home","pts_qtr3_home","pts_qtr4_home",
    # Puntos OT home (pueden existir varias: pts_ot1_home, pts_ot2_home, etc.)
    "pts_ot1_home","pts_ot2_home","pts_ot3_home","pts_ot4_home","pts_ot5_home",
    "pts_home",
    # Puntos por cuartos (away)
    "pts_qtr1_away","pts_qtr2_away","pts_qtr3_away","pts_qtr4_away",
    # Puntos OT away
    "pts_ot1_away","pts_ot2_away","pts_ot3_away","pts_ot4_away","pts_ot5_away",
    "pts_away"
]

# Reducir la tabla
reduce_table(conn, "line_score", line_score_cols)

# Revisar columnas resultantes
display(get_columns(conn, "line_score"))


✅ Tabla 'line_score' reducida a 23 columnas.


,cid,name,type,notnull,dflt_value,pk
0,0,game_id,TEXT,0,None,0
1,1,team_id_home,TEXT,0,None,0
2,2,team_id_away,TEXT,0,None,0
3,3,pts_qtr1_home,TEXT,0,None,0
4,4,pts_qtr2_home,TEXT,0,None,0
5,5,pts_qtr3_home,TEXT,0,None,0
6,6,pts_qtr4_home,TEXT,0,None,0
7,7,pts_ot1_home,INT,0,None,0
8,8,pts_ot2_home,INT,0,None,0
9,9,pts_ot3_home,INT,0,None,0


# game_info

In [30]:
game_info_cols = ["game_id", "game_date", "attendance"]

reduce_table(conn, "game_info", game_info_cols)

# Ver columnas resultantes
display(get_columns(conn, "game_info"))


✅ Tabla 'game_info' reducida a 3 columnas.


,cid,name,type,notnull,dflt_value,pk
0,0,game_id,TEXT,0,None,0
1,1,game_date,NUM,0,None,0
2,2,attendance,INT,0,None,0


# team

In [31]:
team_cols = ["id", "full_name", "abbreviation", "city", "state"]

reduce_table(conn, "team", team_cols)

# Ver columnas resultantes
display(get_columns(conn, "team"))

✅ Tabla 'team' reducida a 5 columnas.


,cid,name,type,notnull,dflt_value,pk
0,0,id,TEXT,0,None,0
1,1,full_name,TEXT,0,None,0
2,2,abbreviation,TEXT,0,None,0
3,3,city,TEXT,0,None,0
4,4,state,TEXT,0,None,0


# player

In [32]:
# 1) Sinónimos posibles en tu base
syn = {
    "id": ["id", "player_id", "person_id"],
    "full_name": ["full_name", "display_first_last", "player_name", "full_name_en"]
}

# 2) Detectar cuáles existen realmente en 'player'
cols_player = get_columns(conn, "player")["name"].tolist()

def pick_existing(candidates, existing):
    for c in candidates:
        if c in existing:
            return c
    return None

id_col = pick_existing(syn["id"], cols_player)
name_col = pick_existing(syn["full_name"], cols_player)

keep = [c for c in [id_col, name_col] if c is not None]
print("Columnas detectadas para conservar en 'player':", keep)

# 3) Reducir la tabla con las columnas detectadas
reduce_table(conn, "player", keep)


Columnas detectadas para conservar en 'player': ['id', 'full_name']
✅ Tabla 'player' reducida a 2 columnas.


['id', 'full_name']

# play_by_play

In [ ]:
# === Reducir 'play_by_play' usando DB_WORK_PATH (sin 'conn') ===
from pathlib import Path
import sqlite3, pandas as pd, os

# 0) Confirmar ruta a la copia
try:
    DB_WORK_PATH
except NameError:
    DB_WORK_PATH = Path("nba_work.sqlite") 

DB_WORK_PATH = Path(DB_WORK_PATH)
if not DB_WORK_PATH.exists():
    raise FileNotFoundError(f"No encuentro la copia: {DB_WORK_PATH.resolve()}")

# Helpers que abren/cerran la conexión internamente
def run_query(q: str) -> pd.DataFrame:
    with sqlite3.connect(DB_WORK_PATH) as cxn:
        return pd.read_sql_query(q, cxn)

def exec_sql(cmds):
    with sqlite3.connect(DB_WORK_PATH) as cxn:
        cur = cxn.cursor()
        for c in cmds:
            cur.execute(c)
        cxn.commit()

def get_columns(table_name: str) -> pd.DataFrame:
    return run_query(f"PRAGMA table_info({table_name});")

def reduce_table(table_name: str, keep_cols):
    existentes = get_columns(table_name)["name"].tolist()
    keep = [c for c in keep_cols if c in existentes]
    if not keep:
        raise ValueError(f"Ninguna columna de {keep_cols} existe en '{table_name}'. "
                         f"Disponibles: {existentes}")
    tmp = f"{table_name}_tmp"
    exec_sql([
        f"DROP TABLE IF EXISTS {tmp};",
        f"CREATE TABLE {tmp} AS SELECT {', '.join(keep)} FROM {table_name};",
        f"DROP TABLE {table_name};",
        f"ALTER TABLE {tmp} RENAME TO {table_name};"
    ])
    print(f"✅ '{table_name}' reducida a {len(keep)} columnas.")

# 1) Sinónimos por campo
syn = {
    "game_id":              ["game_id","gid"],
    "eventnum":             ["eventnum","event_num","event_number"],
    "eventmsgtype":         ["eventmsgtype","event_msg_type","msg_type","event_type"],
    "eventmsgactiontype":   ["eventmsgactiontype","event_msg_action_type","action_type"],
    "periodo":              ["periodo","period","quarter","prd"],
    "pctimestring":         ["pctimestring","pc_time_string","time_remaining","clock","time"],
    "puntuacion":           ["puntuación","puntuacion","score"],
    "margen_puntuacion":    ["margen_de_puntuación","margen_de_puntuacion","scoremargin","score_margin","margin"],
    "player1_id":           ["player1_id","player_1_id","p1_id"],
    "player1_team_id":      ["player1_team_id","player_1_team_id","p1_team_id","team_id1"],
    "player2_id":           ["player2_id","player_2_id","p2_id"],
    "player2_team_id":      ["player2_team_id","player_2_team_id","p2_team_id","team_id2"],
    "player3_id":           ["player3_id","player_3_id","p3_id"],
    "player3_team_id":      ["player3_team_id","player_3_team_id","p3_team_id","team_id3"],
}

# 2) Detectar columnas presentes y construir keep_cols
cols_exist = get_columns("play_by_play")["name"].tolist()

def pick_existing(candidates, existing):
    for c in candidates:
        if c in existing:
            return c
    return None

orden = [
    "game_id","eventnum","eventmsgtype","eventmsgactiontype",
    "periodo","pctimestring","puntuacion","margen_puntuacion",
    "player1_id","player1_team_id","player2_id","player2_team_id","player3_id","player3_team_id"
]

keep_cols = []
for k in orden:
    col = pick_existing(syn[k], cols_exist)
    if col:
        keep_cols.append(col)


# 3) Reducir tabla y mostrar resultado + tamaño de archivo
reduce_table("play_by_play", keep_cols)
display(get_columns("play_by_play"))



✅ 'play_by_play' reducida a 14 columnas.


,cid,name,type,notnull,dflt_value,pk
0,0,game_id,TEXT,0,None,0
1,1,eventnum,INT,0,None,0
2,2,eventmsgtype,INT,0,None,0
3,3,eventmsgactiontype,INT,0,None,0
4,4,period,INT,0,None,0
5,5,pctimestring,TEXT,0,None,0
6,6,score,TEXT,0,None,0
7,7,scoremargin,TEXT,0,None,0
8,8,player1_id,TEXT,0,None,0
9,9,player1_team_id,TEXT,0,None,0


# team_info_common

In [19]:
def get_columns(table_name: str) -> pd.DataFrame:
    return run_query(f"PRAGMA table_info({table_name});")

def reduce_table(table_name: str, keep_cols):
    existentes = get_columns(table_name)["name"].tolist()
    keep = [c for c in keep_cols if c in existentes]
    if not keep:
        raise ValueError(f"Ninguna columna de {keep_cols} existe en '{table_name}'. "
                         f"Disponibles: {existentes}")
    tmp = f"{table_name}_tmp"
    exec_sql([
        f"DROP TABLE IF EXISTS {tmp};",
        f"CREATE TABLE {tmp} AS SELECT {', '.join(keep)} FROM {table_name};",
        f"DROP TABLE {table_name};",
        f"ALTER TABLE {tmp} RENAME TO {table_name};"
    ])
    print(f"✅ '{table_name}' reducida a {len(keep)} columnas.")

# 1) Columnas requeridas + opcionales (se incluyen solo si existen)
keep_candidates = [
    # requeridas
    "team_id", "season_year", "season_id", "team_conference", "team_division", "w", "l", "pct",
    # opcionales
    "conf_rank", "div_rank", "pts_pg", "reb_pg"
]

# 2) Reducir y mostrar resultado
reduce_table("team_info_common", keep_candidates)
display(get_columns("team_info_common"))


✅ 'team_info_common' reducida a 12 columnas.


,cid,name,type,notnull,dflt_value,pk
0,0,team_id,TEXT,0,None,0
1,1,season_year,TEXT,0,None,0
2,2,season_id,TEXT,0,None,0
3,3,team_conference,TEXT,0,None,0
4,4,team_division,TEXT,0,None,0
5,5,w,INT,0,None,0
6,6,l,INT,0,None,0
7,7,pct,REAL,0,None,0
8,8,conf_rank,INT,0,None,0
9,9,div_rank,INT,0,None,0


# common_player_info

In [21]:
def get_columns(table_name: str) -> pd.DataFrame:
    return run_query(f"PRAGMA table_info({table_name});")

def reduce_table(table_name: str, keep_cols):
    existentes = get_columns(table_name)["name"].tolist()
    keep = [c for c in keep_cols if c in existentes]
    if not keep:
        raise ValueError(f"Ninguna columna de {keep_cols} existe en '{table_name}'. "
                         f"Disponibles: {existentes}")
    tmp = f"{table_name}_tmp"
    exec_sql([
        f"DROP TABLE IF EXISTS {tmp};",
        f"CREATE TABLE {tmp} AS SELECT {', '.join(keep)} FROM {table_name};",
        f"DROP TABLE {table_name};",
        f"ALTER TABLE {tmp} RENAME TO {table_name};"
    ])
    print(f"✅ '{table_name}' reducida a {len(keep)} columnas.")

# 1) Candidatas (obligatorias + opcionales)
keep_candidates = [
    # obligatorias
    "person_id", "display_first_last", "position", "season_exp",
    # opcionales
    "height", "weight", "from_year", "to_year"
]

# 2) Reducir y mostrar resultado
reduce_table("common_player_info", keep_candidates)
display(get_columns("common_player_info"))




✅ 'common_player_info' reducida a 8 columnas.


,cid,name,type,notnull,dflt_value,pk
0,0,person_id,TEXT,0,None,0
1,1,display_first_last,TEXT,0,None,0
2,2,position,TEXT,0,None,0
3,3,season_exp,REAL,0,None,0
4,4,height,TEXT,0,None,0
5,5,weight,TEXT,0,None,0
6,6,from_year,REAL,0,None,0
7,7,to_year,REAL,0,None,0


In [23]:
# === Eliminar tablas que no se van a usar ===
to_drop = [
    "draft_combine_stats",
    "draft_history",
    "game_summary",
    "inactive_players",
    "officials",
    "team_details",
    "team_history"
]
# Comprobar tablas restantes
print("\nTablas finales en la base de trabajo:")
print(run_query("SELECT name FROM sqlite_master WHERE type='table';"))



Tablas finales en la base de trabajo:
                 name
0                game
1         other_stats
2          line_score
3           game_info
4                team
5              player
6        play_by_play
7    team_info_common
8  common_player_info


In [29]:
# 0) Rutas 
try:
    DB_PATH
except NameError:
    DB_PATH = Path("nba.sqlite")          

try:
    DB_WORK_PATH
except NameError:
    DB_WORK_PATH = Path("nba_work.sqlite") 

def list_tables(db_path: Path):
    with sqlite3.connect(db_path) as cxn:
        q = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
        return pd.read_sql_query(q, cxn)["name"].tolist()

def table_columns(db_path: Path, table: str) -> int:
    with sqlite3.connect(db_path) as cxn:
        return len(pd.read_sql_query(f"PRAGMA table_info({table});", cxn))

def table_rows(db_path: Path, table: str) -> int:
    with sqlite3.connect(db_path) as cxn:
        return int(pd.read_sql_query(f"SELECT COUNT(*) AS n FROM {table};", cxn).loc[0, "n"])

def db_size_mb(db_path: Path) -> float:
    return os.path.getsize(db_path) / (1024*1024)

print("\n💾 Tamaño de archivo")
print(f"Original: {size_orig:.2f} MB")
print(f"Trabajada: {size_work:.2f} MB  (ahorro: {size_orig - size_work:.2f} MB | {100*(1 - size_work/size_orig):.1f}% de reducción)")



💾 Tamaño de archivo
Original: 2240.74 MB
Trabajada: 3082.05 MB  (ahorro: -841.31 MB | -37.5% de reducción)


In [30]:
# === Resumen final: tablas, filas y columnas de la base reducida ===
def get_tables(db_path):
    with sqlite3.connect(db_path) as cxn:
        q = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
        return pd.read_sql_query(q, cxn)["name"].tolist()

def get_table_info(db_path, table):
    with sqlite3.connect(db_path) as cxn:
        n_filas = pd.read_sql_query(f"SELECT COUNT(*) AS n FROM {table};", cxn).loc[0,"n"]
        n_cols  = len(pd.read_sql_query(f"PRAGMA table_info({table});", cxn))
    return n_filas, n_cols

# Recorrer todas las tablas de la base reducida
resumen = []
for t in get_tables(DB_WORK_PATH):
    filas, cols = get_table_info(DB_WORK_PATH, t)
    resumen.append([t, filas, cols])

df_final = pd.DataFrame(resumen, columns=["tabla","filas","columnas"])
display(df_final)

print("\nTotales globales:")
print(f"Filas en total: {df_final['filas'].sum():,}")
print(f"Columnas en total (sumadas por tabla): {df_final['columnas'].sum():,}")


,tabla,filas,columnas
0,common_player_info,3632,8
1,game,65698,37
2,game_info,58053,3
3,line_score,58053,23
4,other_stats,28271,19
5,play_by_play,13592899,14
6,player,4815,2
7,team,30,5
8,team_info_common,0,12



Totales globales:
Filas en total: 13,811,451
Columnas en total (sumadas por tabla): 123


In [35]:
# === Exportar TODAS las tablas de la base filtrada a CSV (con chunks) ===
from pathlib import Path
import sqlite3, pandas as pd, os, math, time

# 0) Ruta de la base filtrada (copia de trabajo)
try:
    DB_WORK_PATH
except NameError:
    DB_WORK_PATH = Path("nba_work.sqlite")
DB_WORK_PATH = Path(DB_WORK_PATH)

# 1) Carpeta de salida
OUT_DIR = Path("csv_export")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 2) Utilidades
def list_tables(db_path: Path):
    with sqlite3.connect(db_path) as cxn:
        q = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
        return pd.read_sql_query(q, cxn)["name"].tolist()

def count_rows(db_path: Path, table: str) -> int:
    with sqlite3.connect(db_path) as cxn:
        return int(pd.read_sql_query(f"SELECT COUNT(*) AS n FROM {table};", cxn).loc[0, "n"])

def export_table_to_csv(db_path: Path, table: str, out_dir: Path, chunksize: int = 200_000):
    """
    Exporta una tabla a CSV en chunks (para tablas grandes). 
    - Escribe encabezado en el primer chunk y luego agrega sin encabezado.
    """
    t0 = time.time()
    total = count_rows(db_path, table)
    out_file = out_dir / f"{table}.csv"
    if out_file.exists(): out_file.unlink()  # limpiar si ya existía

    if total == 0:
        # crear CSV vacío con solo encabezados
        with sqlite3.connect(db_path) as cxn:
            cols = pd.read_sql_query(f"PRAGMA table_info({table});", cxn)["name"].tolist()
        pd.DataFrame(columns=cols).to_csv(out_file, index=False, encoding="utf-8")
        print(f"✔ {table}: 0 filas → {out_file.name}")
        return

    with sqlite3.connect(db_path) as cxn:
        sql = f"SELECT * FROM {table};"
        first = True
        done = 0
        for chunk in pd.read_sql_query(sql, cxn, chunksize=chunksize):
            chunk.to_csv(out_file, index=False, mode="w" if first else "a",
                         header=first, encoding="utf-8")
            first = False
            done += len(chunk)
            pct = (done / total) * 100
            print(f"  {table}: {done:,}/{total:,} filas ({pct:5.1f}%)", end="\r")
    dt = time.time() - t0
    size_mb = os.path.getsize(out_file) / (1024*1024)
    print(f"\n✔ {table}: {total:,} filas → {out_file.name} ({size_mb:.2f} MB) en {dt:.1f}s")

# 3) Ejecutar exportación para todas las tablas
tables = list_tables(DB_WORK_PATH)
print("Tablas a exportar:", tables, "\n")

for t in tables:
    export_table_to_csv(DB_WORK_PATH, t, OUT_DIR, chunksize=200_000)

print("\n✅ Exportación completa. Archivos en:", OUT_DIR.resolve())


Tablas a exportar: ['common_player_info', 'game', 'game_info', 'line_score', 'other_stats', 'play_by_play', 'player', 'team', 'team_info_common'] 

  common_player_info: 3,632/3,632 filas (100.0%)
✔ common_player_info: 3,632 filas → common_player_info.csv (0.19 MB) en 2.8s
  game: 65,698/65,698 filas (100.0%)
✔ game: 65,698 filas → game.csv (12.61 MB) en 2.6s
  game_info: 58,053/58,053 filas (100.0%)
✔ game_info: 58,053 filas → game_info.csv (2.17 MB) en 0.3s
  line_score: 58,053/58,053 filas (100.0%)
✔ line_score: 58,053 filas → line_score.csv (5.37 MB) en 0.9s
  other_stats: 28,271/28,271 filas (100.0%)
✔ other_stats: 28,271 filas → other_stats.csv (2.43 MB) en 2.1s
  play_by_play: 13,592,899/13,592,899 filas (100.0%)
✔ play_by_play: 13,592,899 filas → play_by_play.csv (785.25 MB) en 132.7s
  player: 4,815/4,815 filas (100.0%)
✔ player: 4,815 filas → player.csv (0.09 MB) en 0.1s
  team: 30/30 filas (100.0%)
✔ team: 30 filas → team.csv (0.00 MB) en 0.1s
✔ team_info_common: 0 filas → t